# Qwen2-VL-7B 이미지 캡셔닝

Qwen2-VL-7B 모델을 사용하여 이미지에 대한 자세한 캡션을 생성합니다.
각 이미지의 특징(크기, 구조, 세부사항)을 설명하는 캡션을 생성합니다.

In [1]:
# 필요한 라이브러리 import
import torch
from transformers import Qwen2VLForConditionalGeneration, AutoProcessor
from PIL import Image
import os
import json
from pathlib import Path
import time

print("✅ 라이브러리 import 완료")

✅ 라이브러리 import 완료


## GPU 환경 확인

GPU가 제대로 인식되고 사용 가능한지 확인합니다.

In [ ]:
import torch

print("=" * 70)
print("🔍 GPU 환경 진단")
print("=" * 70)

# CUDA 사용 가능 여부
print(f"\n✓ CUDA 사용 가능: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    # GPU 정보
    print(f"✓ GPU 개수: {torch.cuda.device_count()}")
    print(f"✓ 현재 GPU: {torch.cuda.current_device()}")
    print(f"✓ GPU 이름: {torch.cuda.get_device_name(0)}")
    
    # 메모리 정보
    total_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
    allocated_memory = torch.cuda.memory_allocated(0) / 1024**3
    reserved_memory = torch.cuda.memory_reserved(0) / 1024**3
    free_memory = total_memory - reserved_memory
    
    print(f"\n📊 GPU 메모리 상태:")
    print(f"   - 전체: {total_memory:.2f} GB")
    print(f"   - 할당됨: {allocated_memory:.2f} GB")
    print(f"   - 예약됨: {reserved_memory:.2f} GB")
    print(f"   - 사용 가능: {free_memory:.2f} GB")
    
    # GPU 사용률 (대략적)
    usage_percent = (reserved_memory / total_memory) * 100
    print(f"   - 사용률: {usage_percent:.1f}%")
    
    # PyTorch 버전
    print(f"\n✓ PyTorch 버전: {torch.__version__}")
    print(f"✓ CUDA 버전: {torch.version.cuda}")
    
    # 텐서 생성 테스트
    print(f"\n🧪 GPU 연산 테스트 중...")
    test_tensor = torch.randn(1000, 1000).cuda()
    result = test_tensor @ test_tensor.T
    print(f"✅ GPU 연산 정상 작동!")
    
    del test_tensor, result
    torch.cuda.empty_cache()
    
else:
    print("\n⚠️  CUDA를 사용할 수 없습니다!")
    print("   CPU 모드로 실행됩니다 (매우 느림)")

print("\n" + "=" * 70)

## 1. Qwen2-VL-7B 모델 로드

Qwen2-VL-7B-Instruct 모델과 프로세서를 로드합니다.

In [2]:
# 모델 설정
model_name = "Qwen/Qwen2-VL-7B-Instruct"

print("🔄 Qwen2-VL-7B 모델 로드 중...")
print("⚠️  첫 실행 시 모델 다운로드로 시간이 소요될 수 있습니다 (~14GB)")

# 프로세서 로드
processor = AutoProcessor.from_pretrained(
    model_name,
    trust_remote_code=True
)

# 모델 로드 (GPU 사용)
model = Qwen2VLForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

device = next(model.parameters()).device
print(f"✅ 모델 로드 완료 (디바이스: {device})")
print(f"✅ GPU 메모리 사용: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")

🔄 Qwen2-VL-7B 모델 로드 중...
⚠️  첫 실행 시 모델 다운로드로 시간이 소요될 수 있습니다 (~14GB)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


✅ 모델 로드 완료 (디바이스: cuda:0)
✅ GPU 메모리 사용: 4.88 GB


## 2. 이미지 캡셔닝 함수 정의

이미지를 분석하여 상세한 캡션을 생성하는 함수입니다.

In [3]:
def generate_detailed_caption(image_path, max_size=512):
    """
    이미지에 대한 상세한 캡션을 생성합니다.
    
    Args:
        image_path: 이미지 파일 경로
        max_size: 이미지 최대 크기 (메모리 절약)
        
    Returns:
        생성된 캡션 문자열
    """
    try:
        start_time = time.time()
        
        # 이미지 로드 (PIL Image 객체로)
        image = Image.open(image_path).convert('RGB')
        
        # 이미지 리사이징 (큰 이미지는 처리 시간이 오래 걸림)
        width, height = image.size
        if max(width, height) > max_size:
            ratio = max_size / max(width, height)
            new_width = int(width * ratio)
            new_height = int(height * ratio)
            image = image.resize((new_width, new_height), Image.LANCZOS)
            print(f"   📐 이미지 크기 조정: {width}x{height} → {new_width}x{new_height}")
        
        # 프롬프트 설정 - 카테고리 + 5가지 특성 설명
        prompt = """Analyze this drawing and provide:

CATEGORY: Identify if this is a drawing of a HOUSE, PERSON, or TREE. Write only one word.

DESCRIPTION: Provide exactly 5 different descriptive sentences, each focusing on a distinct characteristic:
1. Size and scale of objects (e.g., "The tree is large with many branches")
2. Line quality and depth (e.g., "deeply drawn roots")
3. Spatial composition and placement
4. Detail level and complexity
5. Overall impression or mood

Format:
CATEGORY: [HOUSE/PERSON/TREE]
1. [First sentence]
2. [Second sentence]
3. [Third sentence]
4. [Fourth sentence]
5. [Fifth sentence]"""
        
        # 메시지 형식으로 구성
        messages = [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": image,
                    },
                    {
                        "type": "text", 
                        "text": prompt
                    },
                ],
            }
        ]
        
        # 프로세서로 입력 준비
        text = processor.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        
        print(f"   ⏳ 입력 처리 중...")
        
        # 이미지만 추출 (PIL Image 객체)
        inputs = processor(
            text=[text],
            images=[image],
            videos=None,
            padding=True,
            return_tensors="pt",
        )
        inputs = inputs.to(device)
        
        print(f"   🤖 캡션 생성 중...")
        
        # 캡션 생성 (5개 문장을 위해 토큰 수 증가)
        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=200,  # 5개 문장을 위해 증가
                min_new_tokens=50,
                do_sample=False,  # Greedy decoding
                pad_token_id=processor.tokenizer.pad_token_id,
            )
        
        generated_ids_trimmed = [
            out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
        
        caption = processor.batch_decode(
            generated_ids_trimmed,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )[0]
        
        elapsed = time.time() - start_time
        print(f"   ⏱️  처리 시간: {elapsed:.1f}초")
        
        return caption.strip()
        
    except Exception as e:
        print(f"❌ 오류 발생 ({image_path}): {e}")
        import traceback
        traceback.print_exc()
        return None

print("✅ 캡셔닝 함수 정의 완료")

✅ 캡셔닝 함수 정의 완료


## 모델 로딩 후 GPU 확인

모델이 실제로 GPU에 로드되었는지 확인합니다.

In [ ]:
print("=" * 70)
print("🔍 모델 GPU 배치 확인")
print("=" * 70)

# 모델의 각 레이어가 어디에 있는지 확인
device_map = {}
for name, param in model.named_parameters():
    device = str(param.device)
    if device not in device_map:
        device_map[device] = 0
    device_map[device] += 1

print(f"\n📍 모델 파라미터 배치:")
for device, count in device_map.items():
    print(f"   {device}: {count}개 파라미터")

# 현재 GPU 메모리 사용량
if torch.cuda.is_available():
    allocated = torch.cuda.memory_allocated(0) / 1024**3
    reserved = torch.cuda.memory_reserved(0) / 1024**3
    print(f"\n📊 모델 로드 후 GPU 메모리:")
    print(f"   할당됨: {allocated:.2f} GB")
    print(f"   예약됨: {reserved:.2f} GB")
    
    if allocated < 1.0:
        print(f"\n⚠️  경고: GPU 메모리 사용량이 매우 낮습니다 ({allocated:.2f} GB)")
        print(f"   모델이 CPU에 있을 가능성이 있습니다!")
    else:
        print(f"\n✅ 모델이 GPU에 정상적으로 로드되었습니다!")

print("\n" + "=" * 70)

## 3. 이미지 디렉토리 처리

test 폴더의 모든 이미지에 대해 캡션을 생성합니다.

In [4]:
# 이미지 디렉토리 설정
image_dir = r"C:\Users\helen\Desktop\kt cloud tech up\basic_project\test"
output_file = "image_captions_with_category.json"

# 지원하는 이미지 확장자
image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp']

# 이미지 파일 찾기
image_files = []
for ext in image_extensions:
    image_files.extend(Path(image_dir).glob(f'*{ext}'))
    image_files.extend(Path(image_dir).glob(f'*{ext.upper()}'))

image_files = sorted(set(image_files))

print(f"📁 이미지 디렉토리: {image_dir}")
print(f"🖼️  발견된 이미지: {len(image_files)}개\n")

if len(image_files) == 0:
    print("⚠️  이미지 파일이 없습니다!")
else:
    for img in image_files:
        print(f"   - {img.name}")

📁 이미지 디렉토리: C:\Users\helen\Desktop\kt cloud tech up\basic_project\test
🖼️  발견된 이미지: 4개

   - test_나무.JPG
   - test_남자사람.JPG
   - test_여자사람.JPG
   - test_집.JPG


## 4. 캡션 생성 실행

각 이미지에 대해 캡션을 생성하고 결과를 저장합니다.

In [5]:
# 캡션 생성
results = {}

print("\n" + "=" * 70)
print("🎨 캡션 생성 시작")
print("=" * 70 + "\n")

total_start_time = time.time()

for i, image_path in enumerate(image_files, 1):
    print(f"\n[{i}/{len(image_files)}] 처리 중: {image_path.name}")
    
    caption = generate_detailed_caption(str(image_path))
    
    if caption:
        # 카테고리 추출 (CATEGORY: 다음에 오는 단어)
        category = "UNKNOWN"
        if "CATEGORY:" in caption:
            try:
                category_line = caption.split("CATEGORY:")[1].split("\n")[0].strip()
                category = category_line.upper()
            except:
                pass
        
        results[image_path.name] = {
            "path": str(image_path),
            "category": category,
            "caption": caption
        }
        print(f"✅ 완료")
        print(f"📂 카테고리: {category}")
        print(f"📝 캡션: {caption}")
    else:
        print(f"⚠️  캡션 생성 실패")
    
    # GPU 메모리 정리
    torch.cuda.empty_cache()

total_elapsed = time.time() - total_start_time

print("\n" + "=" * 70)
print(f"✨ 총 {len(results)}개 이미지 처리 완료!")
print(f"⏱️  전체 소요 시간: {total_elapsed:.1f}초 (평균: {total_elapsed/len(image_files):.1f}초/이미지)")
print("=" * 70)


🎨 캡션 생성 시작


[1/4] 처리 중: test_나무.JPG
   ⏳ 입력 처리 중...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   🤖 캡션 생성 중...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   ⏱️  처리 시간: 632.1초
✅ 완료
📂 카테고리: TREE
📝 캡션: CATEGORY: TREE
1. The tree is large with many branches.
2. The lines are deeply drawn, especially in the roots.
3. The tree is centrally placed with a clear sky background.
4. The detail level is moderate, with flowers and leaves.
5. The overall impression is serene and natural.

[2/4] 처리 중: test_남자사람.JPG
   ⏳ 입력 처리 중...
   🤖 캡션 생성 중...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   ⏱️  처리 시간: 681.0초
✅ 완료
📂 카테고리: PERSON
📝 캡션: CATEGORY: PERSON
1. The drawing is small in size, focusing on a single figure.
2. The lines are simple and unshaded, indicating a basic outline.
3. The figure is centrally placed, with no additional background elements.
4. There is minimal detail, with no facial features or clothing textures.
5. The overall impression is minimalistic and abstract.

[3/4] 처리 중: test_여자사람.JPG
   ⏳ 입력 처리 중...
   🤖 캡션 생성 중...


The following generation flags are not valid and may be ignored: ['temperature', 'top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


   ⏱️  처리 시간: 802.1초
✅ 완료
📂 카테고리: PERSON
📝 캡션: CATEGORY: PERSON
1. The figure is small in size, suggesting it is a child or a toy.
2. The lines are simple and unrefined, indicating a child's drawing.
3. The figure is centered in the image, with arms and legs spread out.
4. There is minimal detail, with no facial features or clothing details.
5. The overall impression is playful and innocent, evoking a sense of childhood.

[4/4] 처리 중: test_집.JPG
   ⏳ 입력 처리 중...
   🤖 캡션 생성 중...
   ⏱️  처리 시간: 754.0초
✅ 완료
📂 카테고리: HOUSE
📝 캡션: CATEGORY: HOUSE
1. The drawing is small in size, depicting a simple structure.
2. The lines are thin and simple, lacking depth or shading.
3. The house is centrally placed on the page with a clear roof and door.
4. The drawing is minimalistic with few details, focusing on the basic shape.
5. The overall impression is of a basic, childlike representation of a house.

✨ 총 4개 이미지 처리 완료!
⏱️  전체 소요 시간: 2869.4초 (평균: 717.3초/이미지)
   ⏱️  처리 시간: 754.0초
✅ 완료
📂 카테고리: HOUSE
📝 캡션: C

## 5. 결과 저장

생성된 캡션을 JSON 파일로 저장합니다.

In [6]:
# JSON 파일로 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(results, f, indent=2, ensure_ascii=False)

print(f"\n💾 결과 저장 완료: {output_file}")
print(f"📊 총 {len(results)}개 이미지 캡션 저장됨\n")

# 결과 미리보기
print("=" * 70)
print("📋 생성된 캡션 요약")
print("=" * 70)
for filename, data in results.items():
    print(f"\n🖼️  {filename}")
    print(f"   📂 카테고리: {data.get('category', 'UNKNOWN')}")
    print(f"   📝 캡션: {data['caption'][:100]}{'...' if len(data['caption']) > 100 else ''}")


💾 결과 저장 완료: image_captions_with_category.json
📊 총 4개 이미지 캡션 저장됨

📋 생성된 캡션 요약

🖼️  test_나무.JPG
   📂 카테고리: TREE
   📝 캡션: CATEGORY: TREE
1. The tree is large with many branches.
2. The lines are deeply drawn, especially in...

🖼️  test_남자사람.JPG
   📂 카테고리: PERSON
   📝 캡션: CATEGORY: PERSON
1. The drawing is small in size, focusing on a single figure.
2. The lines are simp...

🖼️  test_여자사람.JPG
   📂 카테고리: PERSON
   📝 캡션: CATEGORY: PERSON
1. The figure is small in size, suggesting it is a child or a toy.
2. The lines are...

🖼️  test_집.JPG
   📂 카테고리: HOUSE
   📝 캡션: CATEGORY: HOUSE
1. The drawing is small in size, depicting a simple structure.
2. The lines are thin...
